**Notas generales.**

Modelos:

Tanto gpt-oss como qwen hacen uso de la estrategia de generación en formato harmony. Es decir, primero tienen una sección interna de razonamiento (un tanto esquizofrénico) PREVIO a la respuesta global. Entre más ambiguo sea el prompt más larga es esta sección, por eso prompts como `prompt4` tienen tan buenos resultados. El modelo es capaz de inferir que se le está pasando un ejemplo y logra seguir el formato de buena manera.

Lo malo de esto es que se tienen que tener más de 1000 tokens de generación ANTES de que el modelo produzca los resultados finales. Gpt-oss ya da resultados buenos en 2000 max_new_tokens y muchas veces no llega al límite de generación. Qwen usa una mayor cantidad de tokens, las primeras respuestas que salen son apartir de x > 4300~4600 tokens.

In [ ]:
import torch, re
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(dev)
print(torch.cuda.get_device_name())

path = r''
dataset = pd.read_json(path, lines = True)
dataset.head()

cuda
NVIDIA RTX 5000 Ada Generation


In [ ]:
# Models

gpt_tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
gpt_model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", device_map = 'cuda')

# -------------------------------
qwen_quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype= torch.bfloat16)
qwen_tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-8B')
qwen_model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen3-8B', quantization_config = qwen_quant_config, device_map = 'cuda')

### Logic Prompts (Igual para todos los checkpoints)

In [ ]:
prompt = """
    Translate the following premises from natural language to first order logic. 
    PREMISES:
    All eels are fish.
    No fish are plants.
    Everything displayed in the collection is either a plant or an animal.
    All multicellular animals are not bacteria.
    All animals displayed in the collection are multicellular.
    A sea eel is displayed in the collection.
    The sea eel is an eel or an animal or not a plant.
"""

prompt2 = """
    Given a set of premises description and a question, the task is to parse the problem into first-order logic formulars.
    --------------
    Premises:
    All people who regularly drink coffee are dependent on caffeine. 
    People either regularly drink coffee or joke about being addicted to caffeine. 
    No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
    Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. 
    If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    Predicates:
    Dependent(x) 
    Drinks(x) 
    Jokes(x) 
    Unaware(x)
    Student(x)
    Premises:
    ∀x (Drinks(x) → Dependent(x)) 
    ∀x (Drinks(x) ⊕ Jokes(x))
    ∀x (Jokes(x) → ¬Unaware(x)) 
    (Student(rina) ∧ Unaware(rina)) ⊕ ¬(Student(rina) ∨ Unaware(rina)) 
    ¬(Dependent(rina) ∧ Student(rina)) → (Dependent(rina) ∧ Student(rina)) ⊕ ¬(Dependent(rina) ∨ Student(rina)) 
    --------------
    
    Premises:
    All eels are fish.
    No fish are plants.
    Everything displayed in the collection is either a plant or an animal.
    All multicellular animals are not bacteria.
    All animals displayed in the collection are multicellular.
    A sea eel is displayed in the collection.
    The sea eel is an eel or an animal or not a plant.
    Predicates:
"""


prompt3 = """
    Given a set of premises, the task is to parse the problem and the question into first-order logic formulars.
    The grammar of the first-order logic formular is defined as follows:
    1) logical conjunction of expr1 and expr2: expr1 ∧ expr2
    2) logical disjunction of expr1 and expr2: expr1 ∨ expr2
    3) logical exclusive disjunction of expr1 and expr2: expr1 ⊕ expr2
    4) logical negation of expr1: ¬expr1
    5) expr1 implies expr2: expr1 → expr2
    6) expr1 if and only if expr2: expr1 ↔ expr2
    7) logical universal quantification: ∀x
    8) logical existential quantification: ∃x
    --------------
    Premises:
    All people who regularly drink coffee are dependent on caffeine. People either regularly drink coffee or joke about being addicted to caffeine. No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    Predicates:
    Dependent(x) ::: x is a person dependent on caffeine.
    Drinks(x) ::: x regularly drinks coffee.
    Jokes(x) ::: x jokes about being addicted to caffeine.
    Unaware(x) ::: x is unaware that caffeine is a drug.
    Student(x) ::: x is a student.
    Premises:
    ∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
    ∀x (Drinks(x) ⊕ Jokes(x)) ::: People either regularly drink coffee or joke about being addicted to caffeine.
    ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
    (Student(rina) ∧ Unaware(rina)) ⊕ ¬(Student(rina) ∨ Unaware(rina)) ::: Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. 
    ¬(Dependent(rina) ∧ Student(rina)) → (Dependent(rina) ∧ Student(rina)) ⊕ ¬(Dependent(rina) ∨ Student(rina)) ::: If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    --------------
    
    Premises:
    All eels are fish.
    No fish are plants.
    Everything displayed in the collection is either a plant or an animal.
    All multicellular animals are not bacteria.
    All animals displayed in the collection are multicellular.
    A sea eel is displayed in the collection.
    The sea eel is an eel or an animal or not a plant.
    Predicates:
"""

final_prompt = """
    Given a set of premises, the task is to parse the problem and the question into first-order logic formulars. Answer only with the translated premises.
    The grammar of the first-order logic formular is defined as follows:
    1) logical conjunction of expr1 and expr2: expr1 ∧ expr2
    2) logical disjunction of expr1 and expr2: expr1 ∨ expr2
    3) logical exclusive disjunction of expr1 and expr2: expr1 ⊕ expr2
    4) logical negation of expr1: ¬expr1
    5) expr1 implies expr2: expr1 → expr2
    6) expr1 if and only if expr2: expr1 ↔ expr2
    7) logical universal quantification: ∀x
    8) logical existential quantification: ∃x
    --------------
    Natural Language Premises:
    All people who regularly drink coffee are dependent on caffeine. People either regularly drink coffee or joke about being addicted to caffeine. No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    Predicates:
    Dependent(x) ::: x is a person dependent on caffeine.
    Drinks(x) ::: x regularly drinks coffee.
    Jokes(x) ::: x jokes about being addicted to caffeine.
    Unaware(x) ::: x is unaware that caffeine is a drug.
    Student(x) ::: x is a student.
    FOL Premises:
    ∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
    ∀x (Drinks(x) ⊕ Jokes(x)) ::: People either regularly drink coffee or joke about being addicted to caffeine.
    ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
    (Student(rina) ∧ Unaware(rina)) ⊕ ¬(Student(rina) ∨ Unaware(rina)) ::: Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. 
    ¬(Dependent(rina) ∧ Student(rina)) → (Dependent(rina) ∧ Student(rina)) ⊕ ¬(Dependent(rina) ∨ Student(rina)) ::: If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    --------------
    
    Natural Language Premises:
    {}
    FOL Premises:
"""

### Generation Functions

In [ ]:
def chat_oss(text):

    message = [
    {"role": "system", "content": "Reasoning: high."},
    {"role": "developer", "content": "You translate natural language premises to first-order-logic formulars. Answer only with the translated premises."},
    {"role": "user", "content": text}
    ]

    inputs = gpt_tokenizer.apply_chat_template(
        message,
        add_generation_prompt = False,
        tokenize = True,
        return_dict = True,
        return_tensors = "pt"
    ).to(gpt_model.device)

    outputs = gpt_model.generate(**inputs, max_new_tokens = 2000) 
    # 50, 200, 350, 600, 1000 no.
    answer = gpt_tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:-10])
    try:
        aux = re.search(r'(final<\|message\|>)', answer)
        return answer[aux.end():]
    except:
        print("No hay token de inicio del mensaje final.")


def chat_qwen(msg, thinking):
    """
        mensajes = list ; lista con el diccionario de inputs.
        thinking = bool ; valor booleano que determina si el modelo usa "thinking mode" xd
    """
    text = qwen_tokenizer.apply_chat_template(
    msg,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=thinking 
    )
    inputs = qwen_tokenizer([text], return_tensors='pt').to(dev)
    gen_ids = qwen_model.generate(**inputs, max_new_tokens = 8000) 
    # 350, 750, 1250, 1500, 2000, 4000. No </think>
    # Con 8000 apenas vimos el primter <\think> token nmms XD.
    output_ids = gen_ids[0][len(inputs.input_ids[0]):].tolist()

    try:
        index = len(gen_ids) - gen_ids[::-1].index(151668)
    except ValueError:
        index = 0

    norm_cont = qwen_tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n") 
    
    regex = re.search(r'(</think>)', norm_cont)
    try:
        return norm_cont[regex.end():] 
    except:
        print("No <\\think> token found.")



# ====================================================================================
# ====================================================================================
# ====================================================================================
# ====================================================================================



def translations(model):
    """
    Genera el conjunto de datos de traducción de FOLIO.
    """
    if model == 'gpt':
        message = [
        {"role": "system", "content": "Reasoning: high."},
        {"role": "developer", "content": "You translate natural language premises to first-order-logic formulars. Answer only with the translated premises."},
        {"role": "user", "content": ''}
        ]
    else:
        message = [
            {"role": "user", "content": ''}
        ]

    translation = []
    for _ in dataset["premises"]:
        aux_prompt = final_prompt.format(_)
        message[-1]['content'] = aux_prompt
        if model == 'gpt':
            answer = chat_oss(message)
        else:
            answer = chat_qwen(message)

        translation.append(answer)
    return translation

In [ ]:
gpt_translations = translations('gpt')
qwen_translations =translations('qwen')

dataset['gpt Translations'] = gpt_translations
dataset['qwen Translations'] = qwen_translations

### gpt-oss

#### Lógica Finalmente (ESTÁ CHIDO)

In [ ]:
template_n = [
    {"role": "system", "content": "Reasoning: high."},
    {"role": "developer", "content": "You translate natural language premises to first-order-logic formulars. Answer only with the translated premises."},
    {"role": "user", "content": ''}
]

for texto in prompt_list:
    template_n[2]['content'] = texto
    chat_oss(template_n)
    print("===================================================================================")
    print("===================================================================================")
    print("===================================================================================")

Outputs: torch.Size([2168])
**Predicates**

| Symbol | Meaning |
|--------|---------|
| `Eel(x)`        | *x* is an eel |
| `Fish(x)`       | *x* is a fish |

Outputs: torch.Size([2464])
No hay token de inicio del mensaje final.
Outputs: torch.Size([2435])
**First‑order‑logic representation of the second set of premises**

Let the following predicates be used (constants are written in lower‑case letters in parentheses):

| Predicate | Meaning |
|-----------|---------|
| `Eel(x)` | *x* is an eel |
| `Fish(x)` | *x* is a fish |
| `Plant(x)` | *x* is a plant |
| `Animal(x)` | *x* is an animal |
| `Multicellular(x)` | *x* is multicellular |
| `Bacteria(x)` | *x* is bacteria |
| `Displayed(x)` | *x* is displayed in the collection |

Let the constant **`seaEel`** denote the particular sea eel mentioned in the
text.

Then the premises translate into the following first‑order‑logic formulas:

1. **All eels are fish**  
   \[
   \forall x\; (\,\text{Eel}(x) \;\rightarrow\; \text{Fish}(x)\,)
   

### Qwen3

In [ ]:
def chat_qwen(msg, thinking):
    """
        mensajes = list ; lista con el diccionario de inputs.
        thinking = bool ; valor booleano que determina si el modelo usa "thinking mode" xd
    """
    text = qwen_tokenizer.apply_chat_template(
    msg,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=thinking 
    )
    inputs = qwen_tokenizer([text], return_tensors='pt').to(dev)
    gen_ids = qwen_model.generate(**inputs, max_new_tokens = 8000) 
    # 350, 750, 1250, 1500, 2000, 4000. No </think>
    # Con 8000 apenas vimos el primter <\think> token nmms XD.
    output_ids = gen_ids[0][len(inputs.input_ids[0]):].tolist()
    print(len(output_ids))

    try:
        index = len(gen_ids) - gen_ids[::-1].index(151668)
    except ValueError:
        index = 0

    #thinking_content = qwen_tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    norm_cont = qwen_tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n") 
    
    regex = re.search(r'(</think>)', norm_cont)
    try:
        #print("Normal: ", norm_cont[regex.end():])  
        return norm_cont[regex.end():] 
    except:
        print("No <\\think> token found.")


In [7]:
qwen_msg = [
    {"role": "user", "content": ''}
]

prompt_no = 0
for p in prompt_list:
    prompt_no += 1
    print("Prompt{}".format(prompt_no))
    qwen_msg[0]["content"] = p
    chat_qwen(qwen_msg, True)
    print("===========================================================================================")
    print("===========================================================================================")

Prompt1
5789
Normal:  

Here is the translation of the premises into first-order logic:

1. **All eels are fish.**  
   $ \forall x \, (\text{Eel}(x) \rightarrow \text{Fish}(x)) $

2. **No fish are plants.**  
   $ \forall x \, (\text{Fish}(x) \rightarrow \neg \text{Plant}(x)) $

3. **Everything displayed in the collection is either a plant or an animal.**  
   $ \forall x \, (\text{Displayed}(x) \rightarrow \text{Plant}(x) \vee \text{Animal}(x)) $

4. **All multicellular animals are not bacteria.**  
   $ \forall x \, ((\text{Animal}(x) \wedge \text{Multicellular}(x)) \rightarrow \neg \text{Bacteria}(x)) $

5. **All animals displayed in the collection are multicellular.**  
   $ \forall x \, ((\text{Animal}(x) \wedge \text{Displayed}(x)) \rightarrow \text{Multicellular}(x)) $

6. **A sea eel is displayed in the collection.**  
   $ \exists x \, (\text{SeaEel}(x) \wedge \text{Displayed}(x)) $

7. **The sea eel is an eel or an animal or not a plant.**  
   $ \exists x \, (\text{SeaEel}(

### Deepseek

Nuevamente este modelo hijo de puta pesa 69,000 billones de parámetros y es lo peor del mundo tratar de usarlo desde HuggingFace. Toca usarlo desde otro lugar.

In [ ]:
from lmdeploy import pipeline

pipe = pipeline("deepseek-ai/DeepSeek-V3")

message = [{"role": "user", "content": "Tell me who Lionel Messi is."}]
output = pipe(message)
print(output)

Fetching 185 files:   0%|          | 0/185 [00:00<?, ?it/s]

configuration_deepseek.py: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

niah.png:   0%|          | 0.00/108k [00:00<?, ?B/s]

benchmark.png:   0%|          | 0.00/184k [00:00<?, ?B/s]

...

model-00023-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

model-00021-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

model-00024-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

model-00025-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

model-00026-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]